In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Model

In [2]:
!pip install transformers

In [3]:
from transformers import AutoTokenizer
from transformers import AutoTokenizer, MT5Model, TrainingArguments, Trainer, MT5ForConditionalGeneration, AutoModelForSeq2SeqLM
import torch
import wandb

# tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model_name = "google/mt5-base" #"google/mt5"  # "csebuetnlp/banglat5_nmt_en_bn"Adjust if using a pre-trained model
# model_name = "csebuetnlp/banglat5"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = MT5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(torch_device)

c:\Users\ooga\miniconda3\envs\ml_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

  from .autonotebook import tqdm as notebook_tqdm

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.

You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.

Some weights of MT5ForConditionalGeneration were not initialized from the model checkpoint at weights/checkpoint-8095 and are newly initialized: ['lm_head.weight']

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Dataset

In [4]:
! pip install git+https://github.com/csebuetnlp/normalizer
from normalizer import normalize


  Cloning https://github.com/csebuetnlp/normalizer to c:\users\ooga\appdata\local\temp\pip-req-build-kgv40fsy

  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9

  Preparing metadata (setup.py): started

  Preparing metadata (setup.py): finished with status 'done'






  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer 'C:\Users\ooga\AppData\Local\Temp\pip-req-build-kgv40fsy'


In [5]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val =  pd.read_csv("data/val.csv")

In [6]:
df_train = df_train[:100]
df_val = df_val[:20]

In [7]:
print(df_train.isna().sum())
print(df_test.isna().sum())
print(df_val.isna().sum())

torch.cuda.empty_cache()

id                     0

dataset                0

text_transliterated    0

text_bengali           0

dtype: int64

id                     0

dataset                0

text_transliterated    0

text_bengali           0

dtype: int64

id                     0

dataset                0

text_transliterated    0

text_bengali           0

dtype: int64


## Dataset Preprocessing
### Normalize

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary NLTK resources (may need internet connection)
nltk.download('punkt')
nltk.download('stopwords')

def clean_text(text, language='english'):
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])

    # Convert to lowercase
    text = text.lower()

    # Remove stopwords (optional, adjust stopword list based on language)
    stop_words = stopwords.words(language)
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

# Clean Banglish and Bengali text
# df['Banglish_Clean'] = df['Banglish'].apply(clean_text)
# df['Bengali_Clean'] = df['Bengali'].apply(clean_text, language='bengali')  # Specify Bengali for stopword removal

# Normalization for Bengali text (replace with your desired normalization function)
def normalize_bengali(text):
    normalized_text = normalize(text)
    return normalized_text

df_train['normalized_bengali'] = df_train['text_bengali'].apply(normalize_bengali)
df_test['normalized_bengali'] = df_test['text_bengali'].apply(normalize_bengali)
df_val['normalized_bengali'] = df_val['text_bengali'].apply(normalize_bengali)


[nltk_data] Downloading package punkt to

[nltk_data]     C:\Users\ooga\AppData\Roaming\nltk_data...

[nltk_data]   Package punkt is already up-to-date!

[nltk_data] Downloading package stopwords to

[nltk_data]     C:\Users\ooga\AppData\Roaming\nltk_data...

[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df_train["normalized_bengali"][0]

'ফোন রেজুলেশন ৪৮০পি, ভিডিও রেকর্ডিং ১০৮০পি! হাও?'

### pad and truncate all the dfs

In [10]:
def find_max_length(df, column_name):
    # Find the index of the text with the maximum length
    max_length_index = df[column_name].str.len().idxmax()

    # Get the text with the maximum length
    max_length_text = df.loc[max_length_index, column_name]

    # Print the maximum length and the corresponding text
#     print(f"Index of the text with maximum length: {max_length_index}")
#     print(f"Maximum length: {len(max_length_text)}")
#     print(f"Text with maximum length:\n{max_length_text}")
    return len(max_length_text)

# find_max_length(df_train, 'text_bengali')
# print(df_train['text_bengali'][10454])

In [11]:
from datasets import Dataset
def pad_truncate(df):
    max_length = 200
#     print(max_length)
    bengali_tokenized = tokenizer(df['normalized_bengali'].tolist(), padding="max_length", truncation=True)
#     print(bengali_tokenized)
#     max_length = find_max_length(df, 'text_transliterated')
    banglish_tokenized = tokenizer(df['text_transliterated'].tolist(), padding="max_length", truncation=True)

    dataset = Dataset.from_dict({
        "input_ids": banglish_tokenized["input_ids"],
        "attention_mask": banglish_tokenized["attention_mask"],
        "labels": bengali_tokenized["input_ids"]  # Labels are target language tokens
    })
    
    return dataset

train_dataset = pad_truncate(df_train)
# print(banglish, bengali)
# pad_truncate(df_test)
# pad_truncate(df_val)

In [12]:
# test_dataset['input_ids'][0]

In [13]:
# len(train_dataset['labels'][0])

In [14]:
test_dataset = pad_truncate(df_test)
val_dataset = pad_truncate(df_val)

### Model training

In [15]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
# training_args = TrainingArguments(
#     output_dir="./mt5_banglish_bengali",  # Output directory for checkpoints
#     evaluation_strategy="steps",
#     overwrite_output_dir=True,  # Overwrite existing directory if it exists
#     num_train_epochs=3,  # Adjust based on dataset size and desired accuracy
#     per_device_train_batch_size=2,  # Adjust batch size based on GPU memory
#     save_steps=50,  # Save model checkpoints every 10,000 steps
#     save_total_limit=2,  # Keep only the most recent 2 checkpoints
#     logging_steps=50,  # Log training progress every 500 steps
#     fp16 = True,
#     gradient_accumulation_steps = 6,
#     load_best_model_at_end=True  # Load the best model based on validation metrics
# )

batch_size = 4
args = Seq2SeqTrainingArguments(output_dir="weights",
                        evaluation_strategy="epoch",
                        save_strategy = "epoch",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=5,
                        weight_decay=0.01,
                        save_total_limit=3,
                        predict_with_generate=True,
                        fp16 = False,
                        gradient_accumulation_steps = 6,
                        save_steps = 50,
                        logging_steps = 50,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")

c:\Users\ooga\miniconda3\envs\ml_env\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead

  warnings.warn(


In [16]:
# df_train

In [17]:
# from datasets import Dataset

# dataset = Dataset.from_dict({
#     "input_ids": banglish_tokenized["input_ids"],
#     "attention_mask": banglish_tokenized["attention_mask"],
#     "labels": bengali_tokenized["input_ids"]  # Labels are target language tokens
# })

# train_dataset = dataset.train_test_split(test_size=0.2)  # Split into train and validation sets


In [18]:
# df = pd.DataFrame(dataset)

In [19]:
# df.isna().sum(axis = 0)

In [20]:
!pip install bert-score

In [21]:
from datasets import load_metric

# Load the BERTScore metric
bert_metric = load_metric('bertscore')

C:\Users\ooga\AppData\Local\Temp\ipykernel_14588\1619319013.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate

  bert_metric = load_metric('bertscore')

c:\Users\ooga\miniconda3\envs\ml_env\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/bertscore/bertscore.py

You can avoid this message in future by passing the argument `trust_remote_code=True`.

Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.

  warnings.warn(


In [22]:
def compute_metrics(preds_and_labels):
    preds, labels = preds_and_labels

    # Decode the predictions and labels using the tokenizer, skipping special tokens
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 (masked tokens) in labels with the pad token ID
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # Decode the labels using the tokenizer, skipping special tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute BERTScore using decoded predictions and labels
    result = bert_metric.compute(predictions=decoded_preds, references=decoded_labels, lang='bn')
    
    # Return the BERTScore as a dictionary
    return {
      'BERT F1': np.mean(result['f1']),
      'BERT Precision': np.mean(result['precision']),
      'BERT Recall': np.mean(result['recall'])
  }

In [23]:
from transformers import DataCollatorForSeq2Seq

# Instantiate a Seq2Seq model from the specified checkpoint

# Define a data collator for Seq2Seq tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [24]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [25]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.

wandb: Currently logged in as: mushahid. Use `wandb login --relogin` to force relogin


In [ ]:
trainer.evaluate()

c:\Users\ooga\miniconda3\envs\ml_env\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.

  warnings.warn(

100%|██████████| 375/375 [02:48<00:00,  2.22it/s]


{'eval_loss': 1.0521430969238281,
 'eval_BERT F1': 0.8490847468773524,
 'eval_BERT Precision': 0.8660194059610367,
 'eval_BERT Recall': 0.8342954052686691,
 'eval_runtime': 169.0374,
 'eval_samples_per_second': 8.874,
 'eval_steps_per_second': 2.218,
 'epoch': 4.998970769864141}

In [ ]:
def predict_output(input_sentence):
    input_ids = tokenizer(normalize(input_sentence), return_tensors="pt").input_ids.to("cuda")
    generated_tokens = model.generate(input_ids)
    decoded_tokens = tokenizer.batch_decode(generated_tokens)[0]
    decoded_tokens = normalize(decoded_tokens)

    return decoded_tokens

print("start")
df_val['predictions'] = df_val['text_transliterated'].apply(predict_output)
df_val.to_csv("mT5_val.csv", index=False)
print("complete")
    
# df_test.to_csv("banglaT5_small.csv", index = False)

start


c:\Users\ooga\miniconda3\envs\ml_env\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.

  warnings.warn(


complete


In [ ]:
print("start")
df_test['predictions'] = df_test['text_transliterated'].apply(predict_output)
df_test.to_csv("mT5_test.csv", index=False)
print("complete")

start

complete


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction, closest_ref_length, brevity_penalty
from nltk.tokenize import word_tokenize
import math

# Ensure you have the punkt tokenizer models
nltk.download('punkt')

def calculate_bleu(reference_sentence, candidate_sentence):

  # Tokenize the sentences
    reference = [word_tokenize(reference_sentence)]
    candidate = word_tokenize(candidate_sentence)

    # Define the smoothing function
    smoothing_function = SmoothingFunction().method1

  # Calculate BLEU-1
    bleu1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothing_function)
  # Calculate BLEU-2
    bleu2 = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing_function)
  # Calculate BLEU-3
    bleu3 = sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothing_function)
  # Calculate BLEU-4
    bleu4 = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing_function)

    hyp_len = len(candidate)
    ref_len = len(reference[0])
    closest_ref_len =  closest_ref_length(reference, hyp_len)
    bp = brevity_penalty(closest_ref_len, hyp_len)

    ratio = hyp_len/ref_len

    return bp*pow((bleu1*bleu2*bleu3*bleu4), 1/4), bp, ratio

# Example reference and candidate sentences in Bangla
reference_sentence = "আপনিও শুরু এখন, অ্যাপ নাম সাইজ কই।"
candidate_sentence = "আপনিও শুরু এখন, অ্যাপ নাম সাইজ কই। "
calculate_bleu(reference_sentence, candidate_sentence)

In [ ]:
import nltk
nltk.download('punkt')
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

# Example reference and candidate sentences in Bangla
reference_sentence = "আপনিও শুরু এখন, অ্যাপ নাম সাইজ কই।"
candidate_sentence = "আপনিও শুরু এখন, অ্যাপ নাম সাইজ কই।"

reference = [word_tokenize(reference_sentence)]
candidate = word_tokenize(candidate_sentence)
print(meteor_score(reference, candidate))

In [ ]:
from torchmetrics.text.bert import BERTScore

bertscore = BERTScore()

preds = ["আপনিও শুরু এখন, অ্যাপ নাম সাইজ কই।"]
target = ["আপনিও শুরু এখন, অ্যাপ ইউজার নাম সাইজ কই।"]

print(bertscore(preds, target))

In [ ]:
import nltk
from nltk.util import ngrams

# Function to calculate ROUGE-1, ROUGE-2, and ROUGE-L scores for a pair of texts
def calculate_rouge_scores(reference_tokens, system_tokens):
    def lcs(X, Y):
        m, n = len(X), len(Y)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if X[i - 1] == Y[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1] + 1
                else:
                    dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

        return dp[m][n]

    # Calculate ROUGE-1 (unigram) scores
    reference_unigrams = set(reference_tokens)
    system_unigrams = set(system_tokens)
    overlap_rouge1 = len(reference_unigrams.intersection(system_unigrams))
    precision_rouge1 = overlap_rouge1 / len(system_unigrams)
    recall_rouge1 = overlap_rouge1 / len(reference_unigrams)
    r1_t = 1 if precision_rouge1 + recall_rouge1 == 0 else 0
    f1_rouge1 = 2 * (precision_rouge1 * recall_rouge1) / (precision_rouge1 + recall_rouge1 + r1_t)

    # Calculate ROUGE-2 (bigram) scores
    reference_bigrams = set(ngrams(reference_tokens, 2))
    system_bigrams = set(ngrams(system_tokens, 2))
    overlap_rouge2 = len(reference_bigrams.intersection(system_bigrams))
    precision_rouge2 = overlap_rouge2 / len(system_bigrams)
    recall_rouge2 = overlap_rouge2 / len(reference_bigrams)
    r2_t = 1 if precision_rouge2 + recall_rouge2 == 0 else 1
    f1_rouge2 = 2 * (precision_rouge2 * recall_rouge2) / (precision_rouge2 + recall_rouge2 + r2_t)

    # Calculate ROUGE-L scores
    lcs_length = lcs(reference_tokens, system_tokens)
    precision_rougeL = lcs_length / len(system_tokens)
    recall_rougeL = lcs_length / len(reference_tokens)
    rL_t = 1 if precision_rougeL + recall_rougeL == 0 else 0
    f1_rougeL = 2 * (precision_rougeL * recall_rougeL) / (precision_rougeL + recall_rougeL + rL_t)

    return {
        'ROUGE-1 Precision': precision_rouge1,
        'ROUGE-1 Recall': recall_rouge1,
        'ROUGE-1 F1': f1_rouge1,
        'ROUGE-2 Precision': precision_rouge2,
        'ROUGE-2 Recall': recall_rouge2,
        'ROUGE-2 F1': f1_rouge2,
        'ROUGE-L Precision': precision_rougeL,
        'ROUGE-L Recall': recall_rougeL,
        'ROUGE-L F1': f1_rougeL,
    }

# Function to calculate the average of ROUGE scores for an array of text pairs
def calculate_average_rouge_scores(reference_texts, system_texts):
    total_scores = {
        'ROUGE-1 Precision': 0,
        'ROUGE-1 Recall': 0,
        'ROUGE-1 F1': 0,
        'ROUGE-2 Precision': 0,
        'ROUGE-2 Recall': 0,
        'ROUGE-2 F1': 0,
        'ROUGE-L Precision': 0,
        'ROUGE-L Recall': 0,
        'ROUGE-L F1': 0,
    }

    num_pairs = len(reference_texts)

    for i in range(num_pairs):
        reference_text = reference_texts[i]
        system_text = system_texts[i]

        reference_tokens = nltk.word_tokenize(reference_text)
        system_tokens = nltk.word_tokenize(system_text)

        scores = calculate_rouge_scores(reference_tokens, system_tokens)

        for key, value in scores.items():
            total_scores[key] += value

    # Calculate the average scores
    average_scores = {key: value / num_pairs for key, value in total_scores.items()}
    
    return average_scores

# Example usage with an array of reference and system texts
# reference_texts = test_d_ground
# system_texts = [correct_grammar(test_d_sentence[i],num_return_sequences=2,input_len=test_d_len[i])[0] for i in range(len(test_d_sentence)) ]

reference_texts = ["আপনিও শুরু এখন, অ্যাপ নাম সাইজ কই।", "আপনিও শুরু এখন, অ্যাপ নাম সাইজ কই।"]
system_texts = ["আপনিও শুরু এখন, অ্যাপ ইউজার নাম সাইজ কই।", "আপনিও শুরু এখন, অ্যাপ ইউজার নাম সাইজ কই।"]


average_scores = calculate_average_rouge_scores(reference_texts, system_texts)
print("Average ROUGE Scores:")
for key, value in average_scores.items():
    print(key + ": {:.4f}".format(value))